In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train=pd.read_csv('cs-training.csv',index_col=0)
test=pd.read_csv('cs-test.csv',index_col=0)
train.shape


(150000, 11)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 13.7 MB


In [5]:
train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


# train model

In [5]:
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True}

In [6]:
# X = train[train_cols]
# print(X)
x=train.iloc[:,1:]
y=train.iloc[:,0]
x1, x2, y1, y2 = train_test_split(x, y, test_size=0.1, random_state=99)
dtrain = xgb.DMatrix(x1, y1)
dvalid = xgb.DMatrix(x2, y2)


In [7]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 50, verbose_eval=5)


[0]	train-auc:0.881936	valid-auc:0.804317
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[5]	train-auc:0.955733	valid-auc:0.837804
[10]	train-auc:0.973866	valid-auc:0.839865
[15]	train-auc:0.985775	valid-auc:0.83934
[20]	train-auc:0.992209	valid-auc:0.8365
[25]	train-auc:0.995348	valid-auc:0.835393
[30]	train-auc:0.997397	valid-auc:0.83242
[35]	train-auc:0.998402	valid-auc:0.828729
[40]	train-auc:0.999047	valid-auc:0.827535
[45]	train-auc:0.999388	valid-auc:0.826823
[50]	train-auc:0.999565	valid-auc:0.826372
[55]	train-auc:0.99968	valid-auc:0.825928
Stopping. Best iteration:
[8]	train-auc:0.966712	valid-auc:0.843044



# predict

In [15]:
x_test=xgb.DMatrix(test.iloc[:,1:])
prediction = model.predict(x_test)
print(prediction)

[0.00995635 0.02241796 0.01467466 ... 0.00252992 0.14847213 0.0115795 ]


In [16]:
result=pd.DataFrame({'Id':range(1,len(prediction)+1),'Probability':prediction})

In [21]:
result.to_csv('result.csv',index=False)